## Import libraries

In [1]:
%autosave 0
import numpy as np
from PIL import Image
from mss import mss
import cv2
import time
import pyautogui

from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
import keras.backend as K

Autosave disabled


Using TensorFlow backend.


## Set variables

In [2]:
sct = mss()
monitor={'top':50,'left':60,'width':650,'height':485}

width = 224
height = 224

## Reset GPU memory if resource exhausted error

In [14]:
def limit_mem():
    K.get_session().close()
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))

limit_mem()

## Driving commands

In [3]:
def forward():
    pyautogui.keyDown('w')
    pyautogui.keyUp('w')

In [4]:
def left():
    pyautogui.keyDown('a')
    pyautogui.keyDown('w')
    pyautogui.keyUp('a')
    pyautogui.keyUp('w')

In [5]:
def right():
    pyautogui.keyDown('d')
    pyautogui.keyDown('w')
    pyautogui.keyUp('d')
    pyautogui.keyUp('w')

## Screen grab and process

In [6]:
def grab_screen():
    screen = np.array(sct.grab(monitor))
    screen = cv2.resize(screen, (width, height))
    return screen

## Main function

In [ ]:
# load trained model
model = load_model('keras_models/gtavc1.h5')

print("running")

# pause to change to GTA windows
time.sleep(4)

while(True):

    screen = grab_screen()
    screen_img = cv2.cvtColor(screen, cv2.COLOR_BGR2RGB)
    screen_img = Image.fromarray(screen_img, 'RGB')
    
    x = image.img_to_array(screen_img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    
    prediction = model.predict(x)
    print(prediction)

    max_prediction = prediction.argmax()

    if (max_prediction == 0):
        forward()
        print("forward")
        
    elif (max_prediction == 1):
        left()
        print("left")
        
    elif (max_prediction == 2):
        right()
        print("right")

    cv2.imshow('window', screen)

    if (cv2.waitKey(25) & 0xFF == ord('q')):
        cv2.destroyAllWindows()
        break